<a href="https://colab.research.google.com/github/ingugi/MMAI831-Finance/blob/main/Bonus_Question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#For this question, I need to intialize the companies which have been used in the previous question

#Start with importing libraries

import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
!pip install quandl
import quandl
import scipy.optimize as sco
import yfinance as yf
import pandas as pd

import statsmodels.api as sm
from scipy.optimize import minimize

np.random.seed(42)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Define the stock symbols
stocks = ['AAPL', 'AMZN', 'GOOGL', 'BAC', 'MCD', 'CAT', 'TSLA', 'PFE']

# Create an empty DataFrame to store the data
data = pd.DataFrame()

# Loop through each stock symbol and retrieve the data
for stock in stocks:
    # Retrieve the daily stock data for the year 2022
    stock_data = yf.download(stock, start='2017-01-01', end='2022-12-31')
       
    # Add a column for the stock symbol
    stock_data.loc[:, 'ticker'] = stock
    
    # Append the stock data to the main DataFrame
    data = pd.concat([data, stock_data], axis=0)

# Reset the index of the DataFrame
data.reset_index(inplace=True)

# Select the desired columns
data = data[['Date', 'ticker', 'Adj Close']]

# Rename the columns to match the original code
data.columns = ['date', 'ticker', 'adj_close']

# Print the first few rows of the data
print(data.head())

# Installing IRX T-bill

# Define the start and end dates
start_date = '2017-01-01'
end_date = '2022-12-31'

# Download the risk-free interest rate data
risk_free_data = yf.download('^IRX', start=start_date, end=end_date)

# Print the downloaded data
print(risk_free_data)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
        date ticker  adj_close
0 2017-01-03   AAPL  27.095848
1 2017-01-04   AAPL  27.065519
2 2017-01-05   AAPL  27.203159
3 2017-01-06   AAPL  27.506430
4 2017-01-09   AAPL  27.758368
[*********************100%***********************]  1 of 1 completed
             Open   High    Low  Close  Adj Close  Volume
Date                                                     
2017-01-03  0.488  0.490  0.475  0.475      0.475       0
2017-01-04  0.518  

In [3]:
# keep this 'data' as original and use df. We drop the level(0), which is the adj_closed written on the first line, no need for it
# Set the index of the DataFrame
df = data.set_index('date')

# Pivot the DataFrame to have stocks as columns
table = df.pivot(columns='ticker')
table.columns = table.columns.droplevel(0)

# Calculate the daily percentage returns
returns = table.pct_change()
table.head()


ticker,AAPL,AMZN,BAC,CAT,GOOGL,MCD,PFE,TSLA
date,,,,,,,,
2017-01-03,27.095848,37.683498,19.599291,79.812836,40.400501,102.466888,24.618328,14.466000
2017-01-04,27.065519,37.859001,19.964657,79.456177,40.388500,102.346954,24.834671,15.132667
2017-01-05,27.203159,39.022499,19.729776,78.972153,40.651001,102.535416,25.073399,15.116667
2017-01-06,27.506430,39.799500,19.729776,79.006111,41.260502,103.443405,24.976416,15.267333
2017-01-09,27.758368,39.846001,19.616682,78.437180,41.359001,103.160713,24.968952,15.418667


In [4]:
# Estimate the inputs needed for the MV Analysis
mean_returns = returns.mean()
cov_matrix = returns.cov()
num_portfolios = 2500  # number of random portfolios

# assuming the first day of the year
first_day_2022 = "2022-01-01"

# selecting the weekly data
weekly_returns_2022 = returns.loc[first_day_2022:].resample("W").last()

# estimating the covariance matrix
cov_matrix_2022 = weekly_returns_2022.cov()

# Removing the duplication of the word "ticker" in the covariance matrix
cov_matrix_2022.columns.name = None
cov_matrix_2022.index.name = None

# Printing the covariance matrix for weekly returns in 2022
print(cov_matrix_2022)




           AAPL      AMZN       BAC       CAT     GOOGL       MCD       PFE  \
AAPL   0.000626  0.000550  0.000295  0.000223  0.000507  0.000225  0.000234   
AMZN   0.000550  0.001760  0.000547  0.000359  0.000598  0.000169  0.000210   
BAC    0.000295  0.000547  0.000440  0.000253  0.000302  0.000122  0.000162   
CAT    0.000223  0.000359  0.000253  0.000630  0.000317  0.000094  0.000197   
GOOGL  0.000507  0.000598  0.000302  0.000317  0.000590  0.000183  0.000211   
MCD    0.000225  0.000169  0.000122  0.000094  0.000183  0.000179  0.000180   
PFE    0.000234  0.000210  0.000162  0.000197  0.000211  0.000180  0.000353   
TSLA   0.000639  0.000907  0.000292  0.000278  0.000568  0.000148  0.000161   

           TSLA  
AAPL   0.000639  
AMZN   0.000907  
BAC    0.000292  
CAT    0.000278  
GOOGL  0.000568  
MCD    0.000148  
PFE    0.000161  
TSLA   0.001405  


In [5]:

# Define the list of dates corresponding to the first business day of each week in 2022
start_date = '2022-01-01'
end_date = '2022-12-31'
dates = pd.date_range(start_date, end_date, freq='BMS')  # BMS: business month start

# Create an empty DataFrame to store the portfolio returns
portfolio_returns = pd.DataFrame(index=dates, columns=['Portfolio'])

# Loop through each date and compute portfolio returns
for date in dates:
    # Select the weekly returns data up to the current date
    selected_returns = returns.loc[:date]

    # Calculate the updated covariance matrix using the selected data
    cov_matrix = selected_returns.cov()

    # Calculate the updated expected returns based on the mean of the selected data
    expected_returns = selected_returns.mean()

    # Define the objective function for portfolio optimization (e.g., minimize variance)
    def objective(weights):
        return np.dot(weights, np.dot(cov_matrix, weights))

    # Define the constraints (e.g., weights sum to 1)
    constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]

    # Define the bounds (e.g., weights between 0 and 1)
    bounds = [(0, 1)] * len(stocks)

    # Perform portfolio optimization to find the weights that minimize the objective function
    result = minimize(objective, len(stocks) * [1 / len(stocks)], method='SLSQP', constraints=constraints, bounds=bounds)

    # Get the optimal weights from the optimization result
    weights = result.x

    # Calculate the portfolio return by multiplying the weights with the expected returns
    portfolio_return = np.dot(weights, expected_returns)

    # Store the portfolio return in the DataFrame
    portfolio_returns.loc[date, 'Portfolio'] = portfolio_return

# Print the portfolio returns
print(portfolio_returns)


           Portfolio
2022-01-03  0.001371
2022-02-01  0.001301
2022-03-01  0.001228
2022-04-01   0.00128
2022-05-02  0.001176
2022-06-01   0.00115
2022-07-01  0.001082
2022-08-01  0.001148
2022-09-01    0.0011
2022-10-03  0.001026
2022-11-01  0.001047
2022-12-01  0.001045


In [6]:
# Perform portfolio optimization to find the weights that maximize the Sharpe ratio
def objective(weights):
    # Calculate the portfolio return
    portfolio_return = np.dot(weights, expected_returns)

    # Calculate the portfolio volatility
    portfolio_volatility = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))

    # Calculate the Sharpe ratio
    sharpe_ratio = portfolio_return / portfolio_volatility

    return -sharpe_ratio  # Minimize the negative Sharpe ratio to maximize it


In [7]:
# Set the constraints and bounds for portfolio optimization
constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]  # Weights sum to 1
bounds = [(0, 1)] * len(stocks)  # Weights between 0 and 1

# Perform portfolio optimization to find the weights that maximize the Sharpe ratio
result = minimize(objective, len(stocks) * [1 / len(stocks)], method='SLSQP', constraints=constraints, bounds=bounds)
tangent_weights = result.x

# Calculate the tangent portfolio return
tangent_portfolio_return = np.dot(tangent_weights, expected_returns)

# Calculate the tangent portfolio volatility
tangent_portfolio_volatility = np.sqrt(np.dot(tangent_weights, np.dot(cov_matrix, tangent_weights)))

# Calculate the weekly returns of the tangent portfolio
tangent_portfolio_weekly_returns = weekly_returns_2022.dot(tangent_weights)

# Compute the mean and standard deviation of the weekly returns of the tangent portfolio
mean_tangent_portfolio_returns = tangent_portfolio_weekly_returns.mean()
std_tangent_portfolio_returns = tangent_portfolio_weekly_returns.std()

# Print the results
print("Tangent Portfolio Returns:")
print(tangent_portfolio_weekly_returns)
print("\nMean of Tangent Portfolio Returns:")
print(mean_tangent_portfolio_returns)
print("\nStandard Deviation of Tangent Portfolio Returns:")
print(std_tangent_portfolio_returns)

Tangent Portfolio Returns:
date
2022-01-09   -0.005040
2022-01-16    0.000717
2022-01-23   -0.017975
2022-01-30    0.028494
2022-02-06    0.003637
2022-02-13   -0.017399
2022-02-20   -0.010223
2022-02-27    0.018209
2022-03-06   -0.002942
2022-03-13   -0.006497
2022-03-20    0.015519
2022-03-27    0.002915
2022-04-03    0.000444
2022-04-10   -0.011359
2022-04-17   -0.011702
2022-04-24   -0.024042
2022-05-01   -0.022599
2022-05-08    0.002849
2022-05-15    0.020296
2022-05-22   -0.005605
2022-05-29    0.031547
2022-06-05   -0.026618
2022-06-12   -0.032082
2022-06-19    0.001209
2022-06-26    0.029447
2022-07-03    0.012126
2022-07-10    0.005084
2022-07-17    0.010402
2022-07-24   -0.002548
2022-07-31    0.026095
2022-08-07   -0.015640
2022-08-14    0.026084
2022-08-21   -0.007346
2022-08-28   -0.028882
2022-09-04   -0.015044
2022-09-11    0.018604
2022-09-18   -0.003587
2022-09-25   -0.021201
2022-10-02   -0.017237
2022-10-09   -0.028404
2022-10-16   -0.030915
2022-10-23    0.033938
20

In [8]:
# Select the weekly returns data for the year 2022
weekly_returns_2022 = returns.loc['2022-01-01':'2022-12-31']

# Calculate the mean of the weekly returns for each stock
mean_returns = weekly_returns_2022.mean()

# Calculate the standard deviation of the weekly returns for each stock
std_returns = weekly_returns_2022.std()

# Calculate the equal weights for the portfolio
n_stocks = len(stocks)
weights = np.array([1/n_stocks] * n_stocks)

# Calculate the weekly returns of the equally weighted portfolio
equally_weighted_portfolio_returns = weekly_returns_2022.dot(weights)

# Compute the mean and standard deviation of the weekly returns of the equally weighted portfolio
mean_equally_weighted_portfolio_returns = equally_weighted_portfolio_returns.mean()
std_equally_weighted_portfolio_returns = equally_weighted_portfolio_returns.std()

# Print the results
print("Mean of Equally Weighted Portfolio Returns:")
print(mean_equally_weighted_portfolio_returns)
print("\nStandard Deviation of Equally Weighted Portfolio Returns:")
print(std_equally_weighted_portfolio_returns)


Mean of Equally Weighted Portfolio Returns:
-0.0010429405691128758

Standard Deviation of Equally Weighted Portfolio Returns:
0.017546639124269136


1. Mean of Tangent Portfolio Returns: -0.001031
   - This value represents the average weekly return of the tangent portfolio. It indicates that, on average, the tangent portfolio experienced a small negative return each week in 2022.

2. Standard Deviation of Tangent Portfolio Returns: 0.018353
   - This value represents the measure of volatility or risk associated with the tangent portfolio's returns. A higher standard deviation suggests that the returns of the portfolio had a wider range or were more volatile.

3. Mean of Equally Weighted Portfolio Returns: -0.001043
   - This value represents the average weekly return of the equally weighted portfolio. Similar to the tangent portfolio, it indicates that the equally weighted portfolio had a small negative return on average each week in 2022.

4. Standard Deviation of Equally Weighted Portfolio Returns: 0.017547
   - This value represents the measure of volatility or risk associated with the equally weighted portfolio's returns. Comparing it to the standard deviation of the tangent portfolio, we see that the equally weighted portfolio had a slightly lower volatility or risk.

Both portfolios had similar average returns, with a small negative return on average each week in 2022. The tangent portfolio had a slightly higher standard deviation, indicating slightly higher volatility or risk compared to the equally weighted portfolio. 